# animating dimension reduction of spike times as point processes
Tim Tyree<br>
2.20.2021

### TODO: compute the point process feature vectors in matlab and save as .csv


In [1]:
#automate the boring stuff
from lib.getterdone import *

In [4]:
import h5py
import json,os

In [27]:
#search_for_filename
# input_file_name=search_for_file()
#preparsed json data is deserialized
input_file_name=f"{nb_dir}/Data/Archie/spike_times_trial_4.json"

#manual data has an ugly dtype
# input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_Manual/catTempPopStorage_full.mat'
#early autogen data
# input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_AutoGen/Archie_SRT_Set226_subset1_201020_142509_UnitStorage.mat'


In [121]:
with open(input_file_name) as f:
    data_raw = json.load(f)
# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
type(data_raw)

dict

In [122]:
#initialize a list of length N=number of neurons
N=len(list(data_raw.keys()))
X=np.full(N,np.nan,dtype=np.float64)
#functionally compute the time since last spike time for a given time for each neuron
t=0
for n,key in enumerate(data_raw.keys()):
    spikes=np.array(data_raw[key],np.float64)
    spikes=spikes[spikes<t]
    if not spikes.shape[0]==0:
        lag=t-spikes[-1]
        X[n]=lag #time since last spike in the nth neuron
        # X[n]=1/lag #rate since last spike in the nth neuron

In [125]:
l=0
key_self=list(data_raw.keys())[l]
times_self=np.array(data_raw[key_self],np.float64)
M=times_self.shape[0]

In [126]:
# #represent spiketime trial for 1 neuron as a point process in terms of an 1 x m x n numpy array
arr = np.full([M, N], np.nan)

#initialize a list of length N=number of neurons
N=len(list(data_raw.keys()))
# X=np.full(N,np.nan,dtype=np.float64)
#functionally compute the time since last spike time for a given time for each neuron
for m,t in enumerate(times_self):
    for n,key in enumerate(data_raw.keys()):
        spikes=np.array(data_raw[key],np.float64)
        spikes=spikes[spikes<t]
        if not spikes.shape[0]==0:
            lag=t-spikes[-1]
            arr[m,n]=lag #time since last spike in the nth neuron
            # X[n]=1/lag #rate since last spike in the nth neuron

In [128]:
#TODO: find the biggest subsquare with no nan values
#GOAL: have no nan values in arr
#find the fraction of True for each column
boo_neurons_keep=~np.isnan(arr)[-1,:]
for n in range(N):
    earliest_time_thresh=-1
    if boo_neurons_keep[n]:
        boo=np.isnan(arr)[:,n]
        frac=sum(boo)/boo.shape[0]
        #earliest time of nonnan value for given neuron is less than earliest_time_thresh
        boo_neurons_keep[n]=times_self[~boo][0]<earliest_time_thresh

In [129]:
boo_times_keep=times_self>earliest_time_thresh
Mnew=sum(boo_times_keep)
Nnew=sum(boo_neurons_keep)
arrnew=arr[boo_times_keep][:,boo_neurons_keep]
#assert there are no remaining nan values
assert (not np.isnan(arrnew).any())

In [130]:
data=arrnew

In [131]:
#TODO(later): represent spiketime trial for all neurons as an array of point processes in terms of an l x m x n numpy array


In [132]:
#TODO: naïve eigenvalue decomposition of ^that

#TODO: animate a umap with the number of neighbors \propto time


In [95]:
#(doesn't work)generate example data
nx, ny = (3, 2)
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
for i in range(nx):
    for j in range(ny):
        # treat xv[i,j], yv[i,j]
        xv[i,j]=i; yv[i,j]=j
data=np.multiply(xv,yv)

# visualize different parameter settings with umap

In [135]:
# #load example data
# from sklearn.datasets import load_digits
# digits = load_digits()
# data = digits.data
# data.shape

In [134]:
# #TODO: Represent point process of spike times as an 8x8 bit image (under some reasonable normalization that is consistent accross samples)
# index=212
# plt.subplots(figsize=(1,1))
# plt.imshow(data[index].reshape((8,8)),cmap='gray')
# plt.axis('off')
# plt.show()

In [110]:
# Ignore UserWarnings
import warnings
warnings.simplefilter('ignore', UserWarning)
from umap import UMAP

In [136]:
def tween(e1, e2, n_frames=20):
    for i in range(5):
        yield e1
    for i in range(n_frames):
        alpha = i / float(n_frames - 1)
        yield (1 - alpha) * e1 + alpha * e2
    for i in range(5):
        yield(e2)
    return

In [137]:
def generate_frame_data(data, arg_name='n_neighbors', arg_list=[]):
    result = []
    es = []
    for arg in arg_list:
        kwargs = {arg_name:arg}
        if len(es) > 0:
            es.append(UMAP(init=es[-1], negative_sample_rate=3, **kwargs).fit_transform(data))
        else:
            es.append(UMAP(negative_sample_rate=3, **kwargs).fit_transform(data))
        
    for e1, e2 in zip(es[:-1], es[1:]):
        result.extend(list(tween(e1, e2)))
        
    return result

In [141]:
def create_animation(frame_data, arg_name='n_neighbors', arg_list=[]):
    fig, ax = plt.subplots()
    all_data = np.vstack(frame_data)
    frame_bounds = (all_data[:, 0].min() * 1.1, 
                    all_data[:, 0].max() * 1.1,
                    all_data[:, 1].min() * 1.1, 
                    all_data[:, 1].max() * 1.1)
    ax.set_xlim(frame_bounds[0], frame_bounds[1])
    ax.set_ylim(frame_bounds[2], frame_bounds[3])
    points = ax.scatter(frame_data[0][:, 0], frame_data[0][:, 1], s=5, #c=digits.target, cmap='Spectral', 
                        animated=True)
    title = ax.set_title('', fontsize=24)
    ax.set_xticks([])
    ax.set_yticks([])

#     cbar = fig.colorbar(
#         points,
#         cax=make_axes_locatable(ax).append_axes("right", size="5%", pad=0.05),
#         orientation="vertical",
#         values=np.arange(10),
#         boundaries=np.arange(11)-0.5,
#         ticks=np.arange(10),
#         drawedges=True,
#     )
#     cbar.ax.yaxis.set_ticklabels(np.arange(10), fontsize=18)

    def init():
        points.set_offsets(frame_data[0])
        arg = arg_list[0]
        arg_str = f'{arg:.3f}' if isinstance(arg, float) else f'{arg}'
        title.set_text(f'{arg_name}={arg_str}')
#         title.set_text(f'UMAP with {arg_name}={arg_str}')
        return (points,)

    def animate(i):
        points.set_offsets(frame_data[i])
        if (i + 15) % 30 == 0:
            arg = arg_list[(i + 15) // 30]
            arg_str = f'{arg:.3f}' if isinstance(arg, float) else f'{arg}'
            title.set_text(f'{arg_name}={arg_str}')
#             title.set_text(f'UMAP with {arg_name}={arg_str}')
        return (points,)

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(frame_data), interval=20, blit=True)
    plt.close()
    return anim

In [142]:
def animate_param(data, arg_name='n_neighbors', arg_list=[]):
    frame_data = generate_frame_data(data, arg_name, arg_list)
    return create_animation(frame_data, arg_name, arg_list)

In [145]:
data=arrnew #naive spike dwell times did yield seperable connection types
# data=1/arrnew  #naive spike rates did not yield seperable connection types
animate_param(data, 'n_neighbors', [3, 4, 5, 7, 10, 15, 25, 50, 100, 200])

In [146]:
animate_param(data, 'min_dist', [0.0, 0.01, 0.1, 0.2, 0.4, 0.6, 0.9])

In [147]:
animate_param(data, 'local_connectivity', [0.1, 0.2, 0.5, 1, 2, 5, 10])

In [148]:
animate_param(data, 'set_op_mix_ratio', np.linspace(0.0, 1.0, 10))

In [ ]:
%%time
embedding = umap.UMAP(n_components=2, metric='hellinger').fit(word_doc_matrix)

#embed interactive video toys in your documents!  
# http://localhost:8888/notebooks/umap/notebooks/Document%20embedding%20using%20UMAP.ipynb

- I would expect the Mahalanobis distance to be the natural measure of gaussian distributions embedded in $\mathbb{R}^{+64}$.
- Hellinger distance measures the distance between two probability distributions
- Consider XX[i] as the true distribution of times since each neuron has fired conditioned on the ith neuron having just fired? 
    - Question: what is the hellinger distance between the distributions __before vs. after__ the stimulus onset at time t=0?  
    - Question: what is the hellinger distance between the max liklihood estimate for XX[i] and the uniform distribution UU[i] (as a null hypothesis?)


In [136]:
#for a discussion of the following distance measures, see
# http://localhost:8888/notebooks/umap/notebooks/UMAP%20usage%20and%20parameters.ipynb

### ``metric``

The final UMAP parameter we will be considering in this notebook is the ``metric`` parameter. This controls how distance is computed in the ambient space of the input data. By default UMAP supports a wide variety of metrics, including:

**Minkowski style metrics**
* euclidean
* manhattan
* chebyshev
* minkowski

**Miscellaneous spatial metrics**
* canberra
* braycurtis
* haversine

**Normalized spatial metrics**
* mahalanobis
* wminkowski
* seuclidean

**Angular and correlation metrics**
* cosine
* correlation

**Metrics for binary data**
* hamming
* jaccard
* dice
* russelrao
* kulsinski
* rogerstanimoto
* sokalmichener
* sokalsneath
* yule

Any of which can be specified by setting ``metric='<metric name>'``; for example to use cosine distance as the metric you would use ``metric='cosine'``.

UMAP offers more than this however -- it supports custom user defined metrics as long as those metrics can be compiled in ``nopython`` mode by numba. For this notebook we will be looking at such custom metrics. To define such metrics we'll need numba ...

# TODO: apply naïve umap to a point process representation of one trial.
- TODO: look for a convenient interface to explore .mat files or .hdf5 files.  MATLAB?  Then, how to import to python?  Save to .hdf5 file?

- TODO: import 1 trial of spiketimes as a numpy array
    - N = number of neurons
- TODO: for the i^th neuron, compute the time since last firing for each neuron, X[i]
    - X[i][j,k] = the latest time since the k^th neuron has fired at the time the j^th time the i^th neuron has fired
- TODO: compute at least one interesting UMAP .mp4 for each neuron, i=1,...,N


and then,
- TODO: find two trials that are a match/mismatch pair, 

In [135]:
pip install hdf5plugin

     |████████████████████████████████| 829 kB 3.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
